In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()


  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=b52a718d58259385b92f4646256f97741d72ca5afb3625d8d6782e79826f8ba7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1803 (delta 44), reused 45 (delta 20), pack-reused 1722
Receiving objects: 100% (1803/1803), 128.69 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds(seed=SEED):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds(SEED)

cpu


In [3]:
from data_utils import (PpgDaliaExtractor, FormatPPGDalia)

extractor = PpgDaliaExtractor(DATA_DIR)
ppg_dalia_formatter = FormatPPGDalia()
dfs_train_dalia = [ppg_dalia_formatter.transform(extractor.extract_subject(i)) for i in range(1,16)]
[len(df)//32 for df in dfs_train_dalia]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


[9206,
 8199,
 8735,
 9144,
 9299,
 5244,
 9337,
 8074,
 8554,
 10642,
 9042,
 7908,
 9130,
 8952,
 7933]

In [4]:
from data_utils import (Pamap2Handler, FormatPamap)

SUBJECTS = list(range(1,9))

handler = Pamap2Handler(DATA_DIR)
formatter = FormatPamap()
dfs_train_pamap2 = [formatter.transform(handler.get_protocol_subject(i)) for i in SUBJECTS]
[len(df)//200 for df in dfs_train_pamap2]

download


[1882, 2235, 1264, 1647, 1873, 1809, 1567, 2040]

In [12]:
git_pull()

import importlib

import PPG
import preprocessing_utils
from PPG import FullTrainer
import RegressionHR

from RegressionHR import FullTrainer
from RegressionHR import PceLstmDefaults
from RegressionHR import PceLstmModel
from RegressionHR import TrainerJoint
from RegressionHR import  UtilitiesData
from RegressionHR import FullTrainer2

import Trainer
from  Trainer import BatchTrainers
from Trainer import BatchComputers
from Trainer import Interfaces


importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.NoHrPceLstmModel)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.TrainerIS)
importlib.reload(PPG.FullTrainer)
importlib.reload(PceLstmDefaults)
importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR)
importlib.reload(RegressionHR.FullTrainer)
importlib.reload(RegressionHR.FullTrainer2)
importlib.reload(RegressionHR.PceLstmDefaults)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(preprocessing_utils)
importlib.reload(RegressionHR.TrainerJoint)
importlib.reload(RegressionHR.UtilitiesData)
importlib.reload(RegressionHR.PceLstmModel)

importlib.reload(preprocessing_utils)
importlib.reload(Trainer)
importlib.reload(Trainer.BatchTrainers)
importlib.reload(Trainer.BatchComputers)
importlib.reload(Trainer.ToolBox)
importlib.reload(Trainer.Interfaces )

Already up to date.


<module 'Trainer.Interfaces' from '/tmp/HeartRateRegression/Trainer/Interfaces.py'>

In [13]:
import matplotlib.pyplot as plt
import pickle

def compute_ensemble(results):
  
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]
  min_len_y = min([len(y) for y in ys])
  ys = [y[:min_len_y] for y in ys]
  for i in range(1, len(ys)-1):
    # assert np.all(ys[i] == ys[i-1])
    assert np.all(np.abs(ys[i] - ys[i-1])<1)
  ps = np.stack([v["predictions"][1].reshape(-1).numpy()[:min_len_y] for v in results])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.figure()
  plt.plot(a)
  plt.plot(y)
  plt.show()

  return np.mean(np.abs(a - y)), np.mean(np.abs(ps - y))


def compute_ensemble_named(results, model_name="PceLstm"):
  
  results = [r[model_name] for r in results]

  ys = [v["labels"].reshape(-1) for v in results]
  min_len_y = min([len(y) for y in ys])
  ys = [y[:min_len_y] for y in ys]
  for i in range(1, len(ys)-1):
    # assert np.all(ys[i] == ys[i-1])
    assert np.all(np.abs(ys[i] - ys[i-1])<1)
  ps = np.stack([v["predictions"].reshape(-1)[:min_len_y] for v in results])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.figure()
  plt.plot(a)
  plt.plot(y)
  plt.show()

  return np.mean(np.abs(a - y)), np.mean(np.abs(ps - y))

# compute_ensemble_named(aresults[0].values())

In [ ]:
fchoice = {
    'is_h_size': 64,
    'ts_per_is': 12,
    'period_s': 4,
    'step_s': 2,
    'alpha': 0.90,
    'ts_per_sample': 50,
    'batch_size': 64,
    'weight_decay': 1e-05,
    'lr': 0.005,
    'dropout_rate': 0.15,
    'lstm_input': 128,
    'lstm_size': 64,
    'ts_h_size': 16,
    'alpha': 0.9,
    'margin': 1,
}

discriminator_false_label = 0
from RegressionHR import FullTrainer2
import Trainer.BatchTrainers
from PPG import UtilitiesDataXY
from collections import defaultdict
import torch 

nepoch = 100
aresults_pamap2 = defaultdict(dict)
aresults_dalia = defaultdict(dict)
pamap2_size = 8
dalia_size = 15
for val_sub in range(min(pamap2_size, dalia_size)):
  for ts_sub1 in range(pamap2_size):
    if val_sub == ts_sub1:
      continue
    val_sub1 = val_sub
    val_sub2 = val_sub
    ts_sub2 = ts_sub1

    filename = f"pamap_ts_{ts_sub1}_val_{val_sub1}_dalia_ts_{ts_sub2}_val_{val_sub2}_alpha_{fchoice['alpha']}_lr_{fchoice['lr']}-joint_val_nepoch_{nepoch}_triplet.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      full_trainer = FullTrainer2.PceLstmTripletDiscriminatorPamap2DaliaJointTraining(
        dfs_ds1 = dfs_train_pamap2,
        dfs_ds2 = dfs_train_dalia,
        device = args["device"],
        nepoch = nepoch,
        ts_sub1 = ts_sub1,
        ts_sub2 = ts_sub2,
        val_sub1 = val_sub1,
        val_sub2 = val_sub2,
        main_index = 0
        )
      
      out = full_trainer.train(**fchoice)
      with open(save_path, "wb") as f:
        pickle.dump(out, f)    
    print(out["args"], [out[k]['metric'] for k in (full_trainer.lstm_name1,  full_trainer.lstm_name2, full_trainer.disciminator_name1, full_trainer.disciminator_name2)])
    aresults_pamap2[ts_sub1][val_sub1] = out
    print(f"{ts_sub1}-TS:{compute_ensemble_named(list(aresults_pamap2[ts_sub1].values()), full_trainer.lstm_name2)}")
  for ts_sub2 in range(dalia_size):
    if val_sub == ts_sub2:
      continue
    val_sub1 = val_sub
    val_sub2 = val_sub
    ts_sub1 = ts_sub2 % pamap2_size
    filename = f"dalia_ts_{ts_sub2}_val_{val_sub2}_pamap2_ts_{ts_sub1}_val_{val_sub1}_alpha_{fchoice['alpha']}_lr_{fchoice['lr']}-joint_val_nepoch_{nepoch}_triplet.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      full_trainer = FullTrainer2.PceLstmTripletDiscriminatorPamap2DaliaJointTraining(
        dfs_ds1 = dfs_train_pamap2,
        dfs_ds2 = dfs_train_dalia,
        device = args["device"],
        nepoch = nepoch,
        ts_sub1 = ts_sub1,
        ts_sub2 = ts_sub2,
        val_sub1 = val_sub1,
        val_sub2 = val_sub2,
        main_index = 1
        )
      out = full_trainer.train(**fchoice)
      with open(save_path, "wb") as f:
          pickle.dump(out, f)
    print(out["args"], [out[k]['metric'] for k in (full_trainer.lstm_name1,  full_trainer.lstm_name2, full_trainer.disciminator_name1, full_trainer.disciminator_name2)])
    aresults_dalia[ts_sub2][val_sub2] = out
    print(f"{ts_sub2}-TS:{compute_ensemble_named(list(aresults_dalia[ts_sub1].values()), full_trainer.lstm_name2)}")

ts_per_is: 12
sample_per_ts: 400
ts_per_is: 12
sample_per_ts: 128
best val epoch: 1
[1/100]: loss_train: [20.739332, 18.37647, 0.9984704318260889, 1.0036534539039066] loss_val [18.624647, 13.765243, 0.9722293801089733, 1.0000000001241025] loss_ts [22.344332, 18.485308, 0.9999999998348902, 0.9773606751856528]
best val epoch: 2
[2/100]: loss_train: [20.367867, 15.997773, 0.9989666351099947, 0.9872147343541857] loss_val [18.497616, 14.889048, 0.9583951550590034, 1.0002487411568521] loss_ts [21.957983, 17.788136, 1.0000000018574855, 1.0060467780310083]
best val epoch: 3
[3/100]: loss_train: [18.758627, 16.221977, 0.9981056954941632, 0.9855331510108145] loss_val [17.793236, 10.037525, 0.954131672050484, 1.0002880905849603] loss_ts [20.361998, 15.672395, 0.9999999992570058, 0.9921350446504755]
best val epoch: 5
[5/100]: loss_train: [14.260524, 14.371442, 0.9903202217135486, 0.9661666942184739] loss_val [13.528313, 11.333172, 1.0015133610227431, 1.000277829039033] loss_ts [16.207401, 13.99060

In [ ]:
def compute_metric_mean(values, k):
  return np.mean([v[k]['metric'] for v in values])

ms = dict()

for k,values in aresults.items():
  vals = list(aresults[k].values())
  e, m = compute_ensemble_named(vals)
  t = compute_metric_mean(vals, 'PceDiscriminator')
  print(f"{k}: {e} {m} {t}")
  ms[k] = (e, m, t)

print()
for k,v in ms.items():
  print(k, v)

